In [1]:
import pandas as pd
import re
from annoy import AnnoyIndex
import numpy as np
from sentence_transformers import SentenceTransformer
from scipy.spatial.distance import pdist, squareform

## Исследуем файл dealer

In [2]:
dealer = pd.read_csv("C:/Users/VovAz/OneDrive/Desktop/Проект/Хакатон Просепт/marketing_dealer.csv", sep=';', index_col='id')

In [3]:
display(dealer)

,name
id,
1,Moi_vibor_WB
2,Akson
3,Bafus
5,Castorama
6,Cubatora
7,Komus
9,Megastroy
10,OnlineTrade
11,Petrovich


In [4]:
dealer.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18 entries, 1 to 17
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    18 non-null     object
dtypes: object(1)
memory usage: 288.0+ bytes


## Исследуем файл product

In [5]:
product = pd.read_csv('C:/Users/VovAz/OneDrive/Desktop/Проект/Хакатон Просепт/marketing_product.csv', sep=';', index_col='id')

In [6]:
display(product.sample(5))

,Unnamed: 0,article,ean_13,name,cost,recommended_price,category_id,ozon_name,name_1c,wb_name,ozon_article,wb_article,ym_article,wb_article_td
id,,,,,,,,,,,,,,
180,156,223-5,4.680008e+12,Пенное мыло для дозаторов. С цветочным аромато...,465.74,848.0,33.0,Пенное мыло для дозаторов. С цветочным аромато...,Пенное мыло для дозаторов. С цветочным аромато...,Пенное мыло для дозаторов. С цветочным аромато...,NaN,NaN,NaN,NaN
225,230,0024-3 м,4.610093e+12,"Герметик акриловый цвет Медовый, 3 кг",1251.00,2145.0,25.0,Герметик акриловый для швов для деревянных дом...,"Герметик акриловый цвет Медовый, 3 кг",Герметик акриловый для швов для деревянных дом...,453214675.0,150126205.0,0024-3-м,NaN
392,192,019-5,4.680008e+12,Удалитель плесени FUNGI CLEAN концентрат 1:1 /...,673.00,1604.0,35.0,"Удалитель плесени PROSEPT FUNGI CLEAN, концент...","Удалитель плесени PROSEPT FUNGI CLEAN, концент...","Удалитель плесени PROSEPT FUNGI CLEAN, концент...",189522722.0,149974142.0,019-5,NaN
389,485,050-10,4.680008e+12,"Грунт УНИВЕРСАЛЬНЫЙ, пропиточныйготовый состав...",410.00,908.0,26.0,Грунт универсальный пропиточный для различных ...,Грунт универсальный пропиточный для различных ...,Грунт универсальный пропиточный для различных ...,189522784.0,149699635.0,050-10,NaN
253,405,002-1,4.680008e+12,Антисептик трудновымываемый для наружных работ...,298.00,708.0,20.0,Антисептик для наружных работ PROSEPT EXTERIOR...,Антисептик для наружных работ PROSEPT EXTERIOR...,Антисептик для наружных работ PROSEPT EXTERIOR...,189522709.0,150033502.0,002-1,NaN


In [7]:
# Удаляем лишние столбцы


columns_to_drop = ["Unnamed: 0", "article", "ean_13", "cost", "category_id", "ozon_article", "wb_article",
                   "ym_article", "wb_article_td", "ozon_name", "name_1c", "wb_name"]

# Удаление столбцов из датафрейма
product = product.drop(columns=columns_to_drop)

In [8]:
product.info()

<class 'pandas.core.frame.DataFrame'>
Index: 496 entries, 245 to 52
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   name               494 non-null    object 
 1   recommended_price  491 non-null    float64
dtypes: float64(1), object(1)
memory usage: 11.6+ KB


## Исследуем файл key

In [9]:
key = pd.read_csv('C:/Users/VovAz/OneDrive/Desktop/Проект/Хакатон Просепт/marketing_productdealerkey.csv', sep=';', index_col='id')

In [10]:
display(key.sample(5))

,key,dealer_id,product_id
id,,,
1209,103741,14,290
1857,717999700,17,155
1774,1001472223,5,249
292,200544181,3,77
1666,30420419,1,128


In [11]:
key.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1700 entries, 1 to 2023
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   key         1700 non-null   object
 1   dealer_id   1700 non-null   int64 
 2   product_id  1700 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 53.1+ KB


## Исследуем файл price

In [12]:
price = pd.read_csv('C:/Users/VovAz/OneDrive/Desktop/Проект/Хакатон Просепт/marketing_dealerprice.csv', sep=';', index_col='id')

In [13]:
display(price.sample(5))

,product_key,price,product_url,product_name,date,dealer_id
id,,,,,,
7508,1565304,149.0,NaN,Средство для мытья пола Prosept Multipower 800 мл,2023-07-17,7
8173,15881596,521.0,https://www.vseinstrumenti.ru/product/sredstvo...,"Средство для стирки спортивной одежды, обуви и...",2023-07-17,15
591,716501,189.0,https://akson.ru//p/_0_75_bathkrot_111_075/,"Средство для устранения засоров в трубах 0,75л...",2023-07-11,2
8664,200375984,858.0,https://www.bafus.ru/200375984/,Просепт Ultra антисептик невымываемый для отве...,2023-07-18,3
1220,15881594,201.0,https://www.vseinstrumenti.ru/product/sprej-dl...,Спрей для очистки каминных стекол от сажи и ко...,2023-07-11,15


In [14]:
price.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20416 entries, 2 to 20570
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product_key   20416 non-null  object 
 1   price         20416 non-null  float64
 2   product_url   20182 non-null  object 
 3   product_name  20416 non-null  object 
 4   date          20416 non-null  object 
 5   dealer_id     20416 non-null  int64  
dtypes: float64(1), int64(1), object(4)
memory usage: 1.1+ MB


#Объединяем данные


In [15]:
df = price.merge(dealer, left_on='dealer_id', right_on='id')
df = df.merge(key, left_on='product_key', right_on='key')
df = df.merge(product, left_on='product_id', right_on='id')

In [16]:
df.head(5)

,product_key,price,product_url,product_name,date,dealer_id_x,name_x,key,dealer_id_y,product_id,name_y,recommended_price
0,546227,233.0,https://akson.ru//p/sredstvo_universalnoe_pros...,Средство универсальное Prosept Universal Spray...,2023-07-11,2,Akson,546227,2,12,Универсальное моющее и чистящее средствоUniver...,222.0
1,546227,233.0,https://akson.ru//p/sredstvo_universalnoe_pros...,Средство универсальное Prosept Universal Spray...,2023-07-11,2,Akson,546227,2,12,Универсальное моющее и чистящее средствоUniver...,222.0
2,546227,233.0,https://akson.ru//p/sredstvo_universalnoe_pros...,Средство универсальное Prosept Universal Spray...,2023-07-12,2,Akson,546227,2,12,Универсальное моющее и чистящее средствоUniver...,222.0
3,546227,233.0,https://akson.ru//p/sredstvo_universalnoe_pros...,Средство универсальное Prosept Universal Spray...,2023-07-13,2,Akson,546227,2,12,Универсальное моющее и чистящее средствоUniver...,222.0
4,546227,233.0,https://akson.ru//p/sredstvo_universalnoe_pros...,Средство универсальное Prosept Universal Spray...,2023-07-14,2,Akson,546227,2,12,Универсальное моющее и чистящее средствоUniver...,222.0


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17567 entries, 0 to 17566
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   product_key        17567 non-null  object 
 1   price              17567 non-null  float64
 2   product_url        17355 non-null  object 
 3   product_name       17567 non-null  object 
 4   date               17567 non-null  object 
 5   dealer_id_x        17567 non-null  int64  
 6   name_x             17567 non-null  object 
 7   key                17567 non-null  object 
 8   dealer_id_y        17567 non-null  int64  
 9   product_id         17567 non-null  int64  
 10  name_y             17567 non-null  object 
 11  recommended_price  17567 non-null  float64
dtypes: float64(2), int64(3), object(7)
memory usage: 1.6+ MB


### Удаляем лишние данные.

In [18]:
# Список столбцов, которые нужно удалить
columns_to_drop = ["product_key", "product_url", "date", "name_x", "key", "dealer_id_x"]

# Удаление столбцов из датафрейма
df = df.drop(columns=columns_to_drop)

In [19]:
# Переименовываем столбец
df = df.rename(columns={'name_y':'original_name'})

# Выявляем типичные отличия в наименовании товара

In [20]:
# Установка максимальной ширины столбцов для отображения полного текста
pd.set_option('display.max_columns', None)  # Отображение всех столбцов
pd.set_option('display.max_colwidth', None)  # Отображение полного текста в ячейках
df.sample(20)

,price,product_name,dealer_id_y,product_id,original_name,recommended_price
16512,650.0,"Огнебиозащита для древесины PROSEPT ОГНЕБИО PROF 1, 5 литров, 1-ая группа (007-5)",10,306,"Огнебиозащита ОГНЕБИО PROF l, 1 группа, красный готовый состав / 5 л",970.0
14976,569.0,"Удалитель цемента PROSEPT CEMENT CLEANER ,1Л",6,401,Удалитель цемента CEMENT CLEANER концентрат 1:2 / 1 л,654.0
13731,1023.0,Жидкое моющее средство для стирки чёрных тканей Crystal Black концентрат / 3 л,13,159,Жидкое моющее средство для стирки белых и светлых тканейCrystal Blackконцентрат / 3 л,1070.0
16518,650.0,"Огнебиозащита для древесины PROSEPT ОГНЕБИО PROF 1, 5 литров, 1-ая группа (007-5)",10,306,"Огнебиозащита ОГНЕБИО PROF l, 1 группа, красный готовый состав / 5 л",970.0
6546,339.0,"Аэрозоль д/удаления граффити, маркера, краски PROSEPT Duty Graffiti 400мл",9,130,"Средство для удаления граффити, маркера, краскиDuty Graffiti готовый состав, аэрозоль / 0,4 л",485.0
7552,1096.0,"Гель усиленный для глубокой чистки сантехники Bath Extra. Концентрат, 5л",13,58,Гелеобразное средство усиленного действия для удаления ржавчины и минеральных отложенийBath Extraконцентрат 1:10-1:100 / 5 л,1146.0
980,173.0,"Средстводлямытьястекол,зеркалипластикаProseptOpticCristal0,5л",5,200,"Средство для мытья стекол и зеркалOptic Crystalготовый состав / 0,5 л",182.0
6486,357.0,Защита от плесени FUNGI STOP (готовый состав) 0.5 л PROSEPT 018-05,15,393,"Защита от плесени FUNGI STOP готовый состав / 0,5 л",391.0
15970,244.0,"Просепт Professional Optic Crystal средство для мытья стекол, зеркал и пластика (1 л)",3,201,Средство для мытья стекол и зеркалOptic Crystalготовый состав / 1 л,244.0
15446,1932.0,Просепт Professional Duty Stripper средство для глубокой чистки напольных покрытий (5 л),3,120,Средство для удаления напольных защитных покрытийDuty Stripperконцентрат 1:20-1:50 / 5 л,1932.0


# Выявляем типичные отличия в наименовании товара

In [21]:
# Список столбцов, которые нужно удалить
columns_to_drop = ["price", "dealer_id_y",  "recommended_price"]

# Удаление столбцов из датафрейма
df = df.drop(columns=columns_to_drop)

In [22]:
extra_words = (
    r'(?:готовый\sсостав|для|концентрат|просепт|prosept|средство|невымываемый|гелеобразный|канистра|'
    r'чистящее|спрей|универсальный|универсальная|универсальное|пэт|жидкое|моющее|гель|чистки|'
    r'концентрированное|professional|готовое|superrubber)'
)

def clean_description(text):
    text = text.lower()
    text = re.sub(r'\(\d+\)', '', text)
    text = re.sub(r'(?<=[а-яА-Я])(?=[a-zA-Z])|(?<=[a-zA-Z])(?=[а-яА-Я])', ' ', text)
    text = re.sub(r'\b\d+:\d+\b', '', text)
    text = re.sub(r'\b\d+(?::\d+)?[-:]\d+(?::\d+)?\b', '', text)

    # Замена выражений типа "2 группа" на "2-ая группа"
    text = re.sub(r'(\d+)\sгруппа', r'\1-ая группа', text)

    # Замена объема в литрах и веса в килограммах на миллилитры и граммы
    text = re.sub(r'(\d+[,.]\d+)\s*л', lambda x: f"{int(float(x.group(1).replace(',', '.')) * 1000)} мл", text)
    text = re.sub(r'(\d+[,.]\d+)\s*кг', lambda x: f"{int(float(x.group(1).replace(',', '.')) * 1000)} г", text)

    # Добавление пробелов между цифрами и буквами
    text = re.sub(r'(?<=\d)(?=[а-яА-Яa-zA-Z])', ' ', text)
    text = re.sub(r'(?<=[а-яА-Яa-zA-Z])(?=\d)', ' ', text)

    # Удаление выражений вида "007-10бц"
    text = re.sub(r'\b\d+-\d+[а-яА-Яa-zA-Z]*\b', '', text)

    # Удаление слов, соединенных через "/" или точку
    text = re.sub(r'\b\w+[/\.]\w+(?:[/\.]\w+)?\b', '', text)

    # Удаление избыточных слов
    text = re.sub(extra_words, '', text)
    
    # Удаление пробелов перед первым словом
    text = re.sub(r'^\s*', '', text)

    # Удаление запятых, кавычек, дефисов и других символов
    text = re.sub(r'[,"“”/\.()–;+]', '', text)

    # Удаление обособленных пробелами дефисов
    text = re.sub(r'\s-\s', ' ', text)

    # Замена отдельных выражений
    text = re.sub(r'литров|литра|литр', 'л', text)
    text = re.sub(r'\b(мыло-гель|гель-мыло)\b', 'мыло', text)
    text = re.sub(r'osb\s+proplast', 'osbproplast', text)
    text = re.sub(r'\bкор\b', 'коричневый', text)
    text = re.sub(r'eco\s+ultra', 'ecoultra', text)
    text = re.sub(r'гриля\s+и\s+духовых\s+шкафов|удаления\s+пригоревших\s+жиров', 'жироудалитель', text)
    text = re.sub(r'грунт\s+пропиточный', 'грунтовка', text)
    text = re.sub(r'удалитель\s+высолов', 'очиститель фасадов', text)
    text = re.sub(r'с\s+itrus', 'citrus', text)

    return text

# Применяем функцию к столбцам product_name и original_name
df['product_name_normalized'] = df['product_name'].apply(clean_description)
df['original_name_normalized'] = df['original_name'].apply(clean_description)

In [23]:
df.sample(20, random_state=42)

,product_name,product_id,original_name,product_name_normalized,original_name_normalized
14843,Спрей антимикробное д/душ.кабин/акриловых ванн PROSEPT Bath Uni 500мл концентрат,65,"Универсальный концентрат для санитарных комнат с антимикробным эффектомBath Uni готовый состав / 0,5 л",антимикробное акриловых ванн bath uni 500 мл,санитарных комнат с антимикробным эффектом bath uni 500 мл
15011,Средство для удаления цемента Prosept Cement Cleaner 1 л,401,Удалитель цемента CEMENT CLEANER концентрат 1:2 / 1 л,удаления цемента cement cleaner 1 л,удалитель цемента cement cleaner 1 л
6991,"Жидкое средство для стирки PROSEPT CRYSTAL для стирки спортивной одежды, 3 л",153,"Жидкое моющее средство для стирки спортивной одежды, обуви и пуховиковCrystal Sportконцентрат / 3 л",стирки crystal стирки спортивной одежды 3 л,стирки спортивной одежды обуви и пуховиков crystal sport 3 л
1769,"Шпатлевка для плит OSB PROSEPT Proplast 1,3 кг 081-1",234,"Шпатлевка для плит OSBProplast / 1,3 кг",шпатлевка плит osbproplast 1300 г,шпатлевка плит osbproplast 1300 г
1085,АнтисептикProseptECOULTRAкор5л,275,Антисептик ECO ULTRA коричневый / 5 л,антисептик ecoultra коричневый 5 л,антисептик ecoultra коричневый 5 л
16322,"Антисептик PROSEPT АНТИПЛЕСЕНЬ, 5 литров, против грибка и плесени (025-5)",290,Антисептик против грибка и плесени АНТИПЛЕСЕНЬ готовый состав / 5 л,антисептик антиплесень 5 л против грибка и плесени,антисептик против грибка и плесени антиплесень 5 л
3115,"Жидкое мыло PROSEPT Diona Сitrus Е цитрусовый аромат, 5л, ПЭТ",178,жидкое гель-мыло эконом-класса. C ароматом яблокаDiona Citrus Eготовое средство / 5 л ПЭТ,мыло diona citrus е цитрусовый аромат 5 л,-мыло эконом-класса c ароматом яблока diona citrus e 5 л
3379,Просепт Professional Cooky Grill Gel концентрат гелеобразный для чистки гриля и духовых шкафов (500 мл),74,"Средство для чистки гриля и духовых шкафовCooky Grill Gelконцентрированное средство 1:1 – 1:50 / 0,5 л",cooky grill gel жироудалитель 500 мл,жироудалитель cooky grill gel 500 мл
8256,Просепт Bio Lasur антисептик лессирующий защитно-декоративный (2.7 л),332,"Антисептик лессирующий BiO LASUR / белый люкс / 2,7 л",bio lasur антисептик лессирующий защитно-декоративный 2700 мл,антисептик лессирующий bio lasur белый люкс 2700 мл
2830,"Жироудалитель Prosept 0,5 л спрей",72,"Средство для чистки гриля и духовых шкафовCooky Grill концентрированное средство / 0,55 л",жироудалитель 500 мл,жироудалитель cooky grill 550 мл


In [24]:
print(df[['product_name', 'original_name']].sample(20, random_state=42))

                                                                                                  product_name  \
14843                         Спрей антимикробное д/душ.кабин/акриловых ванн PROSEPT Bath Uni 500мл концентрат   
15011                                                 Средство для удаления цемента Prosept Cement Cleaner 1 л   
6991                              Жидкое средство для стирки PROSEPT CRYSTAL для стирки спортивной одежды, 3 л   
1769                                                      Шпатлевка для плит OSB PROSEPT Proplast 1,3 кг 081-1   
1085                                                                            АнтисептикProseptECOULTRAкор5л   
16322                                Антисептик PROSEPT АНТИПЛЕСЕНЬ, 5 литров, против грибка и плесени (025-5)   
3115                                             Жидкое мыло PROSEPT Diona Сitrus Е цитрусовый аромат, 5л, ПЭТ   
3379   Просепт Professional Cooky Grill Gel концентрат гелеобразный для чистки гриля и д

In [25]:
df.drop_duplicates(inplace = True)

In [26]:
sentences = ["огнебиозащита древесины огнебио prof 2-ая группа 65 л", "жироудалитель cooky grill gel 500 мл"]
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
embeddings = model.encode(sentences)
print(embeddings)

[[-1.31915241e-01  4.10210311e-01  2.78960112e-02  1.25279188e-01
   7.74415443e-03 -3.34682688e-02  4.71067391e-02  2.00923204e-01
  -7.11289272e-02  8.19252580e-02 -3.56695789e-04 -1.26996279e-01
  -5.99121638e-02  9.60499793e-02 -8.63214508e-02  2.19218925e-01
  -2.21684650e-01  1.79666728e-01  7.62181506e-02 -3.24606597e-02
   1.61568955e-01 -4.57076468e-02  6.96478337e-02 -1.39384180e-01
  -7.53543004e-02 -1.47356227e-01 -1.70470029e-01  1.21576309e-01
   6.24469221e-02 -1.59005105e-01  8.61100554e-02 -1.16529688e-01
  -1.88719690e-01  1.04947053e-01  9.82760638e-02 -1.25215694e-01
   9.00876522e-03  3.10725868e-02 -5.88298552e-02  9.53392237e-02
  -1.44651145e-01  6.95157722e-02  7.54931718e-02  5.23548499e-02
  -2.47536432e-02  4.58077043e-02 -7.59487972e-02 -3.67418826e-01
  -1.23082258e-01 -9.62827429e-02 -8.01855773e-02 -4.61447351e-02
  -1.97084278e-01  7.73975030e-02  1.89085320e-01 -1.88480943e-01
  -1.97099615e-02  1.15140483e-01 -3.77527252e-02  9.14792642e-02
   2.84032

Тестируем модель на рандомных данных

In [27]:
array = df['product_name_normalized'].values #приводим данные в список
vectors = model.encode(array)#проводим векторизацию
vectors.shape #проверяем

(1690, 384)

In [28]:
array_2 = df['original_name_normalized'].values
vectors_2 = model.encode(array_2)
vectors_2.shape

(1690, 384)

In [29]:
df['original_name_embeddings'] = vectors.tolist()
df['product_name_embeddings'] = vectors_2.tolist()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1690 entries, 0 to 17564
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   product_name              1690 non-null   object
 1   product_id                1690 non-null   int64 
 2   original_name             1690 non-null   object
 3   product_name_normalized   1690 non-null   object
 4   original_name_normalized  1690 non-null   object
 5   original_name_embeddings  1690 non-null   object
 6   product_name_embeddings   1690 non-null   object
dtypes: int64(1), object(6)
memory usage: 105.6+ KB


In [30]:
# Функция для расчета Recall at K
def recall_at_k(predicted, correct, k=5):
    return int(correct in predicted[:k])

In [31]:
unique_original_embeddings = df.drop_duplicates(subset=['product_id'])[['product_id', 'original_name_embeddings']]
unique_embeddings_matrix = np.stack(unique_original_embeddings['original_name_embeddings'].values)

# Ранжирование товаров и подсчет Recall at 5
total_recall = 0
for index, row in df.iterrows():
    # Расчет расстояний
    distances = pdist(np.vstack([row['product_name_embeddings'], unique_embeddings_matrix]), 'cosine')
    distance_matrix = squareform(distances)[0, 1:]
    top_5_indices = np.argsort(distance_matrix)[:5]  # Топ-5 индексов с наименьшими расстояниями

    # Получаем ID топ-5 товаров
    top_5_products_ids = unique_original_embeddings.iloc[top_5_indices]['product_id'].values

    # Добавляем recall для текущего товара
    total_recall += recall_at_k(top_5_products_ids, row['product_id'])

average_recall = total_recall / len(df)
print(f"Средний Recall на Топ-5: {average_recall}")

Средний Recall на Топ-5: 0.8644970414201183


### Заготовка под оптимизацию выдачи результатов

In [32]:
dimension = len(vectors[0])
annoy_index = AnnoyIndex(dimension, metric='angular')

In [33]:
for i, vec in enumerate(vectors):
    annoy_index.add_item(i, vec)

In [34]:
annoy_index.build(n_trees=50)

True

In [35]:
similarities = [annoy_index.get_nns_by_vector(vec, 5) for vec in vectors_2]

In [36]:
df['nearest_neighbors'] = similarities
